### Çözüm 3: Tahmine Dayalı Atama (İşlemleri) ile Doldurma
* DİKKAT: Makine öğrenmesi yöntemleriyle eksiklikleri doldurma konusunu ele alacağız ama henüz makine öğrenmesi öğrenmedik. Dolayısıyla bu konu biraz ileri seviye kalmaktadır. Konuyla ilgili bir miktar makine öğrenmesiyle ilgili fikri olanlar daha keyifli ilerleyebilir. Konuyla ilgili fikri olmayanlar genel mantığı anlamaya odaklanabilir.
* Bir makine öğrenmesi yöntemi ile tahmine dayalı bir şekilde modelleme işlemi gerçekleştireceğiz. Eksikliğe sahip olan değişkeni bağımlı değişken diğer değişkeneleri bağımsız değişkenler gibi kabul edip bir modelleme işlemi gerçekleştireceğiz. Modelleme işlemine göre eksik değerlere sahip olan noktaları tahmin etmeye odaklanacağız. Fakat burada birkaç kritik konu olacak:
* Birincisi kategorik değişkenleri one hot encoder'a sokmamız gerekiyor. Yani bir modelleme tekniği kullanacak olduğumuzdan dolayı bu modelin bizden değişkenleri beklediği bir standart var bundan dollayı bu standarda uymamız gerekmektedir.
* İkincisi ise knn uzaklık temelli bir algoritma olduğundan dolayı değişkenleri standartlaştırmamız gerekiyor.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
def load():
    data = pd.read_csv("titanic.csv")
    return data

df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    
    # önce kategorik değişkenler seçilmiştir. num_but_cat cat_cols'un içerisindedir.
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    
    # numerik ama kategorik olanlar seçilmiştir. 
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    
    # kategorik ama numerik olanlar seçilmiştir.
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    
    # cat_cols listemizi baştan oluşturduk.
    cat_cols = cat_cols + num_but_cat
    
    # Kardinalitesi yüksek olan değişkenleri de bu kısımda temizliyoruz.
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    # tipi objectten farklı olanları getirdik. Yani int veya float
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    
    # Ama oorada numerik olarak gözüken ama kategorik olanlar (saklananlar) vardı. Onlardan da çıkarırız.
    # Böylece elimizde gerçek numerikler gerçek kategorikler olacak.
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    
    # Alttaki kısım sadece raporlama için print edilmiştir. num_but_cat olanlar zaten cat_cols içerisindedir.
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

In [4]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 891
Variables: 12
cat_cols: 6
num_cols: 3
cat_but_car: 3
num_but_cat: 4


In [5]:
cat_cols

['Sex', 'Embarked', 'Survived', 'Pclass', 'SibSp', 'Parch']

In [6]:
num_cols

['PassengerId', 'Age', 'Fare']

In [7]:
cat_but_car

['Name', 'Ticket', 'Cabin']

In [12]:
num_cols = [col for col in num_cols if col not in "PassengerId"]
num_cols

['Age', 'Fare']

In [14]:
# Buradaki cat_cols'a bir dönüşüm işlemi yapmamız gerekiyor, bir encoderda geçirelim...
# Label encoder işlemi ya da one hot encoding işlemi yapmamız gerekiyor. 
# Bu ikisini aynı anda yapabilmek için one hot encoder'ı uygulayacağımız get_dummies metodunu uygulayabiliriz.
dff = pd.get_dummies(df[cat_cols + num_cols], drop_first=True)
# drop_first'ünü True yaparsak bu durumda iki sınıfa sahip olan kategorik değişkenlerin ilk sınıfını tutacak daha doğrusu ilk 
# sınıfını atacak ikinci sınıfını tutacak. Örneğin elimizde cinsiyet gibi male female bir kategorik değişken olduğunda bu 
# kategorik değişkeni de binary bir şekilde temsil edebiliyor olacağız.
dff.head()

# cat_cols + num_cols diyerek iki listeyi topladık...
# get_dummies() metodu bütün değişkenleri birlikte versek de sadece kategorik değişkenlere bir dönüşüm uygulamaktadır. 
# Dolayısıyla kullanacak olduğumuz değişkenleri bir araya getirmeyi tercih ediyoruz. Dikkatimizi çekmiş olacaktır ki 
# cat_cut_car'leri dışarıda bıraktık. Onları istemiyoruz çünkü onların şu anda bir bilgi taşıma özellikleri yok. Kimdi bunlar? 
# name, ticket, cabin değişkenleriydi.

,Survived,Pclass,SibSp,Parch,Age,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,1,0,22.0,7.2500,1,0,1
1,1,1,1,0,38.0,71.2833,0,0,0
2,1,3,0,0,26.0,7.9250,0,0,1
3,1,1,1,0,35.0,53.1000,0,0,1
4,0,3,0,0,35.0,8.0500,1,0,1


In [16]:
# Değişkenlerin standartlaştırılması:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dff = pd.DataFrame(scaler.fit_transform(dff), columns=dff.columns)
dff.head()

,Survived,Pclass,SibSp,Parch,Age,Fare,Sex_male,Embarked_Q,Embarked_S
0,0.0,1.0,0.125,0.0,0.271174,0.014151,1.0,0.0,1.0
1,1.0,0.0,0.125,0.0,0.472229,0.139136,0.0,0.0,0.0
2,1.0,1.0,0.000,0.0,0.321438,0.015469,0.0,0.0,1.0
3,1.0,0.0,0.125,0.0,0.434531,0.103644,0.0,0.0,1.0
4,0.0,1.0,0.000,0.0,0.434531,0.015713,1.0,0.0,1.0


In [17]:
# knn'in uygulanması:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
dff = pd.DataFrame(imputer.fit_transform(dff), columns=dff.columns)
dff.head()

# Bu bize makine öğrenmesi ile tahmine dayalı bir şekilde eksik değerleri doldurma imkanı sağlayacaktır. Model nesnemizi 
# oluşturuyoruz, komşuluk sayımızı 5 yapıyoruz.

,Survived,Pclass,SibSp,Parch,Age,Fare,Sex_male,Embarked_Q,Embarked_S
0,0.0,1.0,0.125,0.0,0.271174,0.014151,1.0,0.0,1.0
1,1.0,0.0,0.125,0.0,0.472229,0.139136,0.0,0.0,0.0
2,1.0,1.0,0.000,0.0,0.321438,0.015469,0.0,0.0,1.0
3,1.0,0.0,0.125,0.0,0.434531,0.103644,0.0,0.0,1.0
4,0.0,1.0,0.000,0.0,0.434531,0.015713,1.0,0.0,1.0


###### KNN Yöntemi Nasıl Çalışır?
* Bana arkadaşını söyle sana kim olduğunu söyleyeyim der...
* Burada şöyle uygulayacağız, mesela yaş değişkeninde eksiklik varya yaş değişkeninde dolu olan diğer gözlemlerden eksik olan değişkene en yakın olan 5 (5'den farklı da olabilir.) komşusunun yaş ortalamasını alır, eksik yaş değişkenindeki değere bulunan sonucu koyar. 

* DİKKAT: Aslında eksiklikleri doldurduk ve işlem bitti ama çirkinleşmeye devam edelim. İyi de biz bu doldurduğumuz yerleri görmek istersek ne yapmalıyız?
* HAYAT KURTARAN SERİSİNDEN BİRKAÇ İŞLEM YAPALIM:
* Öncelikle bu doldurduğumuz değerlerde de bir problem var aslında. Bunları standartlaştırmıştık ya, şu anda değerlerin gerçek değerleri yok. Yani bunlar standartlaştırılmış değerler üzerinden üretilen sonuçlar.İlk problemimiz bu aslında. Biz kıyaslamaya gideceğiz ama daha kıyaslamaya gitmeden önce bu standartlaştırma işleni inverse_transform diyerek geri almalıyız.

In [21]:
dff = pd.DataFrame(scaler.inverse_transform(dff), columns = dff.columns)

In [22]:
dff.head()

,Survived,Pclass,SibSp,Parch,Age,Fare,Sex_male,Embarked_Q,Embarked_S
0,0.0,1.0,0.125,0.0,0.271174,0.014151,1.0,0.0,1.0
1,1.0,0.0,0.125,0.0,0.472229,0.139136,0.0,0.0,0.0
2,1.0,1.0,0.000,0.0,0.321438,0.015469,0.0,0.0,1.0
3,1.0,0.0,0.125,0.0,0.434531,0.103644,0.0,0.0,1.0
4,0.0,1.0,0.000,0.0,0.434531,0.015713,1.0,0.0,1.0


In [23]:
df["age_imputed_knn"] = dff[["Age"]]

In [24]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_imputed_knn
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.271174
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.472229
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.321438
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.434531
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.434531


In [25]:
# Şimdi öyle birşey yapmalıyız ki ilk dataframe'de age ve atanmış age'ler var. Dolayısıyla ilk dataframe'deki "Age" isnull 
# olanları satırlardan seç bu iki değişkeni getir bakalım nereye ne atamışsın diyebiliriz. Ya da bütün değerleri seçebiliriz. 
# Bir yapalım bakalım....

df.loc[df["Age"].isnull(), ["Age", "age_imputed_knn"]]

,Age,age_imputed_knn
5,NaN,0.595376
17,NaN,0.467203
19,NaN,0.148027
26,NaN,0.406886
28,NaN,0.215883
...,...,...
859,NaN,0.318924
863,NaN,0.105303
868,NaN,0.308872
878,NaN,0.301332


In [26]:
# Şu anda bir makine öğrenmesi yöntemi kullanarak eksik değerlerin yerine tahmin edilmiş değerleri atamış olduk...

In [30]:
# tüm değişkenleri detaylı olarak incelemek istersek, Acaba nasıl atanmış görmek istersek:
df.loc[df["Age"].isnull()]
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_imputed_knn
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0.271174
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.472229
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.321438
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.434531
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0.434531
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0.334004
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0.233476
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0.273687
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0.321438
